<a href="https://colab.research.google.com/github/saptarshidatta96/MTech_Sem1/blob/main/Principal%20Component%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
random.seed(42)
%matplotlib inline
import random
import sys
inFile = sys.argv[1]
#outFile = sys.argv[2]

In [ ]:
df = pd.read_csv(inFile, header=None)

In [ ]:
ground_truth = df.iloc[:,4]
data = np.array(df.iloc[:, 0:4]).T

In [ ]:
def encode_ground_truth(ground_truth):
  ground_truth_encoded = []
  for i in range(len(ground_truth)):
    if ground_truth[i] == 'Iris-setosa':
      ground_truth_encoded.append(1)
    elif ground_truth[i] == 'Iris-versicolor':
      ground_truth_encoded.append(2)
    else:
      ground_truth_encoded.append(3)

  return ground_truth_encoded

In [ ]:
def overall_accuracy(cluster):
  ground_truth_encoded = encode_ground_truth(ground_truth)
  df_confusion = confusion_matrix(ground_truth_encoded, cluster) 
  #print(df_confusion)
  sum_diag = 0
  for i in range(len(df_confusion)):
    sum_diag = df_confusion[i][i] + sum_diag
  overall_acc = sum_diag/ df.shape[0]
  print('Accuracy : ', overall_acc)
  #return overall_acc

Principal Component Analysis

In [ ]:
def pca(data, dim):
  val, vec = np.linalg.eig(np.matmul(data, data.T))
  index_sorted_eig = np.argsort(val)[::-1][:dim]
  U = vec[:,index_sorted_eig]

  return U

Spectral Clustering

In [ ]:
def init_centroids(listt):
  initial_centroids = []
  for i in random.sample(range(0, len(listt)), 3):
    initial_centroids.append(listt[i])
  initial_centroids = np.array(initial_centroids)
  #print(initial_centroids)

  return initial_centroids

def K_means(listt, centroids):
  assigned_cluster = []
  for i in range(len(listt)):
    dist1 = (np.sqrt(np.sum((listt[i] - centroids[0])**2)))**2
    #print(dist1)
    dist2 = (np.sqrt(np.sum((listt[i] - centroids[1])**2)))**2
    #print(dist2)
    dist3 = (np.sqrt(np.sum((listt[i] - centroids[2])**2)))**2
    if dist1 < dist2 and dist1 < dist3:
      assigned_cluster.append(1)
    elif dist2 < dist1 and dist2 < dist3:
      assigned_cluster.append(2)
    else:
      assigned_cluster.append(3)

  return assigned_cluster

def update_centroids(listt, predicted_cluster):
  initial_centroids = init_centroids(listt)
  new_centroids = []
  new_data = pd.concat([pd.DataFrame(listt), pd.DataFrame(predicted_cluster, columns=['cluster'])], axis=1)
  #print(new_data)
  for c in set(new_data['cluster']):
    cluster_data = new_data.loc[new_data['cluster'] == c][new_data.columns[:-1]]
    #print(cluster_data)
    cluster_mean = cluster_data.mean(axis = 0)
    #print(cluster_mean)
    new_centroids.append(cluster_mean)

  return new_centroids

def pred_k_means(listt):
  initial_centroids = init_centroids(listt)
  predicted_cluster = K_means(listt, initial_centroids)
  for i in range(50):
    new_centroids = update_centroids(listt, predicted_cluster)
    predicted_cluster = K_means(listt, new_centroids)
  return new_centroids, predicted_cluster

def calc_matrices(data_list):

  W = np.empty((len(data_list), len(data_list)), dtype = 'float')
  for i in range(len(data_list)):
    for j in range(len(data_list)):
      W[i, j] = np.exp(-(np.sqrt(np.sum((data_list[i] - data_list[j])**2)))**2 / 2)
      if i == j:
        W[i, j] = 0

  D = np.diag(W.sum(axis = 1))
  L = D - W
  e, v = np.linalg.eig(L)
  index_sorted_eig = np.argsort(e)[:3]
  H = v[:,index_sorted_eig]

  return H


In [ ]:
def reconstruction_error(U, data_reduced_dim):
  reconstructed_data = np.matmul(U, data_reduced_dim)
  error = np.linalg.norm(data - reconstructed_data)**2
  print('Reconstruction Error:', error)

Applying Spectral Clustering on Ground Truth

In [ ]:
print('------Accuracy without PCA------')
H = calc_matrices(data.T)
new_centroids_spc, predicted_cluster_spc = pred_k_means(H)
accuracy = overall_accuracy(predicted_cluster_spc)

------Accuracy without PCA------
Accuracy :  0.64


PCA with 1 dimension


In [ ]:
print('------PCA with 1 dimension------')
U1 = pca(data, 1)
data_reduced_dim = np.matmul(U1.T, data)
H1 = calc_matrices(data_reduced_dim.T)
new_centroids_spc1, predicted_cluster_spc1 = pred_k_means(H1)
overall_accuracy(predicted_cluster_spc1)
reconstruction_error(U1, data_reduced_dim)


------PCA with 1 dimension------
Accuracy :  0.0
Reconstruction Error: 329.6694039329031


PCA with 2 dimension

In [ ]:
print('------PCA with 2 dimension------')
U2 = pca(data, 2)
data_reduced_dim = np.matmul(U2.T, data)
H2 = calc_matrices(data_reduced_dim.T)
new_centroids_spc2, predicted_cluster_spc2 = pred_k_means(H2)
overall_accuracy(predicted_cluster_spc2)
reconstruction_error(U2, data_reduced_dim)

------PCA with 2 dimension------
Accuracy :  0.7
Reconstruction Error: 15.566331012128215


PCA with 3 dimensions

In [ ]:
print('------PCA with 3 dimension------')
U3 = pca(data, 3)
data_reduced_dim = np.matmul(U3.T, data)
H3 = calc_matrices(data_reduced_dim.T)
new_centroids_spc3, predicted_cluster_spc3 = pred_k_means(H3)
overall_accuracy(predicted_cluster_spc3)
reconstruction_error(U3, data_reduced_dim)

------PCA with 3 dimension------
Accuracy :  0.6333333333333333
Reconstruction Error: 3.5303116664124956
